In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [2]:
df = pd.read_csv('footprint_16_summary_results.csv')
df.head()

,country_code,country,year,EF_percap,quality_score
0,2,Afghanistan,1961,1.21,6
1,2,Afghanistan,1962,1.16,6
2,2,Afghanistan,1963,1.20,6
3,2,Afghanistan,1964,1.22,6
4,2,Afghanistan,1965,1.26,6


In [3]:
df = df.drop(columns=['country_code','country','year'])


In [4]:
df.quality_score.value_counts()

5     3152
6     2874
3B    1450
3T     333
Name: quality_score, dtype: int64

In [5]:
df.isna().sum()

EF_percap        0
quality_score    0
dtype: int64

In [6]:
df['quality_score'].unique()

array(['6', '5', '3B', '3T'], dtype=object)

In [7]:
df['quality_score'] = df['quality_score'].replace(['5'], '3B',)
df.quality_score.value_counts()

3B    4602
6     2874
3T     333
Name: quality_score, dtype: int64

In [8]:
df['quality_score'] = df['quality_score'].replace(['6'], '3T',)
df.quality_score.value_counts()

3B    4602
3T    3207
Name: quality_score, dtype: int64

In [9]:
df_3T = df[df.quality_score=='3T']
df_3B = df[df.quality_score=='3B'].sample(350)
data_df = df_3T.append(df_3B)


In [10]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.quality_score.value_counts()

3T    3207
3B     350
Name: quality_score, dtype: int64

In [11]:
x = data_df.drop(columns='quality_score')
y = data_df['quality_score']
print(x.shape)
print(y.shape)

(3557, 1)
(3557,)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)


In [13]:
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_test, drop_first=True)
x_train.shape

(2489, 1)

In [14]:
def score_data(y_test,pred_y, clf):
    print('\n\n'+clf)
    print("\n CONFUSION MATRIX \n", confusion_matrix(y_test,pred_y))
    print(classification_report(y_test,pred_y))
    print("Accuracy Score is: ", accuracy_score(y_test,pred_y))

In [15]:
LR = LogisticRegression()
LR.fit(x_train, y_train)
pred_y = LR.predict(x_test)
score_data(y_test, pred_y, 'Logistic Regression')



Logistic Regression

 CONFUSION MATRIX 
 [[  0 106]
 [  0 962]]
              precision    recall  f1-score   support

          3B       0.00      0.00      0.00       106
          3T       0.90      1.00      0.95       962

    accuracy                           0.90      1068
   macro avg       0.45      0.50      0.47      1068
weighted avg       0.81      0.90      0.85      1068

Accuracy Score is:  0.900749063670412


C:\Users\David\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\David\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\David\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
x_train.columns = ['x0']
x_test.columns = ['x0']

In [17]:
XG = xgb.XGBClassifier()
XG.fit(x_train, y_train)
pred_y = XG.predict(x_test)
score_data(y_test, pred_y, 'XGB Classifier')

C:\Users\David\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:13:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGB Classifier

 CONFUSION MATRIX 
 [[  2 104]
 [  2 960]]
              precision    recall  f1-score   support

          3B       0.50      0.02      0.04       106
          3T       0.90      1.00      0.95       962

    accuracy                           0.90      1068
   macro avg       0.70      0.51      0.49      1068
weighted avg       0.86      0.90      0.86      1068

Accuracy Score is:  0.900749063670412
